# Data Scraping

In this notebook, we will fetch and download all the necessary datasets and store them in their associated files within the "data" file

### Importing Libraries and Functions

In [2]:
import sys
sys.path.append('../')
import os
from json import dump
from scripts.datascrape_functions import generate_url_list, fetch_rental_data
from scripts.scrape_sa2 import get_shapefile

### Defining Global Variables - changes pending

In [2]:
# CONSTANTS
BASE_URL = "https://www.domain.com.au"
N_PAGES = 2 # RIGHT NOW THIS ONLY LOOKS AT THE FIRST 3 PAGES FOR QUICK RUN TIME THIS NEEDS TO BE UPDATED TO
            # SEARCH ALL THE PAGES!
LANDING_PATH = "data/landing"
RAW_PATH = "data/raw"

### Fetching the '*domain.com*' Rental Property Data

In [3]:
# Generating the list of urls to fetch data from
url_links = generate_url_list(N_PAGES, BASE_URL)

# Fetching the rental data from the domain.comm urls
property_metadata = fetch_rental_data(url_links)


Generating the list of links...

Visiting https://www.domain.com.au/rent/VIC/?sort=price-desc&page=1
Visiting https://www.domain.com.au/rent/VIC/?sort=price-desc&page=2

Fetching the rental data...



100% successful:   0%|          | 0/39 [00:00<?, ?it/s]


In [4]:
# Creates the directory for the rental data if it doesn't exist
if not os.path.exists(f"../{LANDING_PATH}"):
    os.makedirs(f"../{LANDING_PATH}")

# Save the data in the data/landing directory
with open(f'../{LANDING_PATH}/testing_scraping.json', 'w') as f:
    dump(property_metadata, f)
    print(f"File saved in the {LANDING_PATH} directory")


File saved in the data/landing directory


In [3]:
# scrapes SA2 shapfile and downloads to data folder

get_shapefile(url="https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip", output_dir = '../data/SA2/')